In [1]:
# import dependencies
import os
import requests
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from config import mapbox_token
from calendar import monthrange
from config import api_key



%reload_ext lab_black

px.set_mapbox_access_token(mapbox_token)

ModuleNotFoundError: No module named 'config'

In [ ]:
# Column names of main Dataframe
column_names = [
    "speciesCode",
    "comName",
    "sciName",
    "locId",
    "locName",
    "obsDt",
    "howMany",
    "lat",
    "lng	obsValid",
    "obsReviewed",
    "locationPrivate",
    "subId",
    "State",
]

# List of States
list_states = [
    "US-AL",
    "US-AK",
    "US-AZ",
    "US-AR",
    "US-CA",
    "US-CO",
    "US-CT",
    "US-DE",
    "US-DC",
    "US-FL",
    "US-GA",
    "US-HI",
    "US-ID",
    "US-IL",
    "US-IN",
    "US-IA",
    "US-KS",
    "US-KY",
    "US-LA",
    "US-ME",
    "US-MD",
    "US-MA",
    "US-MI",
    "US-MN",
    "US-MS",
    "US-MO",
    "US-MT",
    "US-NE",
    "US-NV",
    "US-NH",
    "US-NJ",
    "US-NM",
    "US-NY",
    "US-NC",
    "US-ND",
    "US-OH",
    "US-OK",
    "US-OR",
    "US-PA",
    "US-RI",
    "US-SC",
    "US-SD",
    "US-TN",
    "US-TX",
    "US-UT",
    "US-VT",
    "US-VA",
    "US-WA",
    "US-WV",
    "US-WI",
    "US-WY",
]

# Global Dataframe
g_dataframe = pd.DataFrame(columns=column_names)

# Each interation do a API call
for state in list_states:
    url = f"https://api.ebird.org/v2/data/obs/{state}/recent"
    r = requests.get(url, params={"key": api_key, "back": 1})
    df_obsdata = pd.DataFrame(r.json())

    # Attach the State code to the data frame
    df_obsdata["State"] = state

    # Append the local data to main data frame that has the state codes
    g_dataframe = pd.concat([g_dataframe, df_obsdata])

In [ ]:
g_dataframe

In [ ]:
# Group by State
g_dataframe.groupby("State").sum()["howMany"]
obs_by_state = g_dataframe.groupby("State").sum()["howMany"]
obs_by_state

In [ ]:
state_sum = pd.DataFrame(obs_by_state)
state_sum

In [ ]:
state_sum = state_sum.reset_index()
state_sum

In [ ]:
state_sum["State_2D"] = state_sum["State"].str[3:]
state_sum

In [ ]:
# state_sum = pd.read_csv(
# "https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv")

fig = go.Figure(
    data=go.Choropleth(
        locations=state_sum["State_2D"],  # Spatial coordinates
        z=state_sum["howMany"].astype(float),  # Data to be color-coded
        locationmode="USA-states",  # set of locations match entries in `locations`
        colorscale="rainbow",
        colorbar_title="Observations",
    )
)

fig.update_layout(
    title_text="US States Bird Sightings ", geo_scope="usa",  # limite map scope to USA
)

fig.show()